In [53]:
import pandas as pd
import numpy as np

In [66]:
with open('2023_day4.csv', 'r') as f:
    s = f.readlines()
    df = pd.DataFrame(s)
df.head()

,0
0,Card 1: 66 90 67 76 55 13 91 31 95 4 | 82 9...
1,Card 2: 94 57 54 45 9 78 71 35 48 44 | 9 5...
2,Card 3: 44 10 67 65 62 43 47 13 26 95 | 67 5...
3,Card 4: 47 48 14 30 64 31 81 22 65 60 | 85 2...
4,Card 5: 43 66 44 76 67 27 52 26 83 39 | 37 8...


In [67]:
a = df[0].str.split(':', expand=True).rename(columns={0:'id'})
a = pd.concat([a['id'], a[1].str.split('|', expand=True)], axis=1)
a[0] = a[0].str.strip().str.replace('  ',' ').str.split(' ')
a[1] = a[1].str.strip().str.replace('  ',' ').str.replace('\n','').str.split(' ')
a['id'] = a['id'].str.replace('Card ','').astype(int)
a.head(10)

,id,0,1
0,1,"[66, 90, 67, 76, 55, 13, 91, 31, 95, 4]","[82, 98, 69, 8, 15, 2, 32, 24, 99, 56, 46, 65,..."
1,2,"[94, 57, 54, 45, 9, 78, 71, 35, 48, 44]","[9, 56, 28, 57, 55, 95, 32, 48, 76, 47, 94, 35..."
2,3,"[44, 10, 67, 65, 62, 43, 47, 13, 26, 95]","[67, 55, 65, 68, 26, 77, 41, 43, 94, 13, 62, 2..."
3,4,"[47, 48, 14, 30, 64, 31, 81, 22, 65, 60]","[85, 23, 47, 19, 60, 31, 32, 68, 64, 33, 14, 2..."
4,5,"[43, 66, 44, 76, 67, 27, 52, 26, 83, 39]","[37, 83, 99, 29, 48, 60, 62, 2, 31, 27, 4, 65,..."
5,6,"[38, 34, 59, 52, 10, 14, 57, 98, 73, 90]","[77, 32, 21, 16, 15, 57, 30, 97, 43, 47, 91, 3..."
6,7,"[94, 57, 27, 45, 70, 34, 83, 30, 51, 17]","[89, 30, 44, 29, 17, 41, 70, 87, 83, 45, 51, 6..."
7,8,"[64, 82, 99, 10, 20, 76, 25, 78, 83, 48]","[66, 62, 98, 32, 48, 2, 79, 97, 65, 70, 89, 56..."
8,9,"[93, 49, 20, 77, 14, 86, 89, 28, 54, 82]","[61, 67, 64, 46, 28, 89, 82, 25, 49, 5, 92, 37..."
9,10,"[50, 76, 67, 32, 84, 46, 54, 30, 42, 74]","[42, 82, 85, 2, 74, 67, 23, 48, 96, 30, 73, 79..."


In [29]:
def match(d):
    matches = 0
    for x in d[1]:
        if x in d[0]:
            matches += 1
    return matches

In [43]:
def nxt(d):
    return [x for x in range(d['id']+1, d['id']+d['res']+1)]


In [51]:
def nxtlvl(d, l, c):
    r = []
    for x in d[f'lvl{l}']:
        for y in c[c['id'] == x][f'lvl1'].iloc[0]:
            r.append(y)
    return r

## This took 50 minutes

probably could do something with nxtlvl function so it uses more array math.

In [68]:
b = a.copy()
b['res'] = b.apply(match, axis=1)
b['lvl1'] = b.apply(nxt, axis=1)
i = 2
empty = False
while not empty:
    b[f'lvl{i}'] = b.apply(lambda x: nxtlvl(x, i-1, b[['id', f'lvl1']]), axis=1)
    b['leng'] = b[f'lvl{i}'].apply(lambda x: len(x))
    if b['leng'].sum() == 0:
        empty = True
    i += 1
b.head(10)

,id,0,1,res,lvl1,lvl2,leng,lvl3,lvl4,lvl5,...,lvl15,lvl16,lvl17,lvl18,lvl19,lvl20,lvl21,lvl22,lvl23,lvl24
0,1,"[66, 90, 67, 76, 55, 13, 91, 31, 95, 4]","[82, 98, 69, 8, 15, 2, 32, 24, 99, 56, 46, 65,...",0,[],[],0,[],[],[],...,[],[],[],[],[],[],[],[],[],[]
1,2,"[94, 57, 54, 45, 9, 78, 71, 35, 48, 44]","[9, 56, 28, 57, 55, 95, 32, 48, 76, 47, 94, 35...",10,"[3, 4, 5, 6, 7, 8, 9, 10, 11, 12]","[4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 5, 6, 7, 8,...",0,"[5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 6, 7, 8, 9...","[6, 7, 8, 9, 7, 8, 9, 10, 11, 12, 13, 14, 15, ...","[7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 9, 1...",...,"[18, 19, 19, 19, 20, 20, 19, 20, 20, 20, 19, 2...","[19, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 2...",[20],[],[],[],[],[],[],[]
2,3,"[44, 10, 67, 65, 62, 43, 47, 13, 26, 95]","[67, 55, 65, 68, 26, 77, 41, 43, 94, 13, 62, 2...",10,"[4, 5, 6, 7, 8, 9, 10, 11, 12, 13]","[5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 6, 7, 8, 9...",0,"[6, 7, 8, 9, 7, 8, 9, 10, 11, 12, 13, 14, 15, ...","[7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 9, 1...","[8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 9, 10, ...",...,"[19, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20]",[20],[],[],[],[],[],[],[],[]
3,4,"[47, 48, 14, 30, 64, 31, 81, 22, 65, 60]","[85, 23, 47, 19, 60, 31, 32, 68, 64, 33, 14, 2...",10,"[5, 6, 7, 8, 9, 10, 11, 12, 13, 14]","[6, 7, 8, 9, 7, 8, 9, 10, 11, 12, 13, 14, 15, ...",0,"[7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 9, 1...","[8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 9, 10, ...","[9, 10, 11, 12, 13, 14, 11, 12, 13, 14, 15, 16...",...,[20],[],[],[],[],[],[],[],[],[]
4,5,"[43, 66, 44, 76, 67, 27, 52, 26, 83, 39]","[37, 83, 99, 29, 48, 60, 62, 2, 31, 27, 4, 65,...",4,"[6, 7, 8, 9]","[7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 9, 1...",0,"[8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 9, 10, ...","[9, 10, 11, 12, 13, 14, 11, 12, 13, 14, 15, 16...","[10, 11, 12, 13, 14, 11, 12, 13, 14, 15, 16, 1...",...,[],[],[],[],[],[],[],[],[],[]
5,6,"[38, 34, 59, 52, 10, 14, 57, 98, 73, 90]","[77, 32, 21, 16, 15, 57, 30, 97, 43, 47, 91, 3...",1,[7],"[8, 9, 10, 11, 12, 13, 14, 15, 16, 17]",0,"[9, 10, 11, 12, 13, 14, 11, 12, 13, 14, 15, 16...","[10, 11, 12, 13, 14, 11, 12, 13, 14, 15, 16, 1...","[11, 12, 13, 14, 15, 16, 13, 14, 15, 15, 16, 1...",...,[],[],[],[],[],[],[],[],[],[]
6,7,"[94, 57, 27, 45, 70, 34, 83, 30, 51, 17]","[89, 30, 44, 29, 17, 41, 70, 87, 83, 45, 51, 6...",10,"[8, 9, 10, 11, 12, 13, 14, 15, 16, 17]","[9, 10, 11, 12, 13, 14, 11, 12, 13, 14, 15, 16...",0,"[10, 11, 12, 13, 14, 11, 12, 13, 14, 15, 16, 1...","[11, 12, 13, 14, 15, 16, 13, 14, 15, 15, 16, 1...","[13, 14, 15, 15, 16, 17, 16, 17, 18, 17, 18, 1...",...,[],[],[],[],[],[],[],[],[],[]
7,8,"[64, 82, 99, 10, 20, 76, 25, 78, 83, 48]","[66, 62, 98, 32, 48, 2, 79, 97, 65, 70, 89, 56...",1,[9],"[10, 11, 12, 13, 14]",0,"[11, 12, 13, 14, 15, 16, 13, 14, 15, 15, 16, 17]","[13, 14, 15, 15, 16, 17, 16, 17, 18, 17, 18, 1...","[14, 15, 15, 16, 17, 16, 17, 18, 16, 17, 18, 1...",...,[],[],[],[],[],[],[],[],[],[]
8,9,"[93, 49, 20, 77, 14, 86, 89, 28, 54, 82]","[61, 67, 64, 46, 28, 89, 82, 25, 49, 5, 92, 37...",5,"[10, 11, 12, 13, 14]","[11, 12, 13, 14, 15, 16, 13, 14, 15, 15, 16, 17]",0,"[13, 14, 15, 15, 16, 17, 16, 17, 18, 17, 18, 1...","[14, 15, 15, 16, 17, 16, 17, 18, 16, 17, 18, 1...","[15, 16, 17, 16, 17, 18, 16, 17, 18, 17, 18, 1...",...,[],[],[],[],[],[],[],[],[],[]
9,10,"[50, 76, 67, 32, 84, 46, 54, 30, 42, 74]","[42, 82, 85, 2, 74, 67, 23, 48, 96, 30, 73, 79...",6,"[11, 12, 13, 14, 15, 16]","[13, 14, 15, 15, 16, 17, 16, 17, 18, 17, 18]",0,"[14, 15, 15, 16, 17, 16, 17, 18, 16, 17, 18, 1...","[15, 16, 17, 16, 17, 18, 16, 17, 18, 17, 18, 1...","[16, 17, 18, 17, 18, 18, 19, 17, 18, 18, 19, 1...",...,[],[],[],[],[],[],[],[],[],[]


In [62]:
def tot(d, i):
    v = 0
    for j in range(1, i-1):
        v += len(d[f'lvl{j}'])
    return v

In [69]:
print(i)
b['tot'] = b.apply(lambda x: tot(x, i), axis=1)
b.head()

25


,id,0,1,res,lvl1,lvl2,leng,lvl3,lvl4,lvl5,...,lvl16,lvl17,lvl18,lvl19,lvl20,lvl21,lvl22,lvl23,lvl24,tot
0,1,"[66, 90, 67, 76, 55, 13, 91, 31, 95, 4]","[82, 98, 69, 8, 15, 2, 32, 24, 99, 56, 46, 65,...",0,[],[],0,[],[],[],...,[],[],[],[],[],[],[],[],[],0
1,2,"[94, 57, 54, 45, 9, 78, 71, 35, 48, 44]","[9, 56, 28, 57, 55, 95, 32, 48, 76, 47, 94, 35...",10,"[3, 4, 5, 6, 7, 8, 9, 10, 11, 12]","[4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 5, 6, 7, 8,...",0,"[5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 6, 7, 8, 9...","[6, 7, 8, 9, 7, 8, 9, 10, 11, 12, 13, 14, 15, ...","[7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 9, 1...",...,"[19, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 2...",[20],[],[],[],[],[],[],[],23735
2,3,"[44, 10, 67, 65, 62, 43, 47, 13, 26, 95]","[67, 55, 65, 68, 26, 77, 41, 43, 94, 13, 62, 2...",10,"[4, 5, 6, 7, 8, 9, 10, 11, 12, 13]","[5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 6, 7, 8, 9...",0,"[6, 7, 8, 9, 7, 8, 9, 10, 11, 12, 13, 14, 15, ...","[7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 9, 1...","[8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 9, 10, ...",...,[20],[],[],[],[],[],[],[],[],11896
3,4,"[47, 48, 14, 30, 64, 31, 81, 22, 65, 60]","[85, 23, 47, 19, 60, 31, 32, 68, 64, 33, 14, 2...",10,"[5, 6, 7, 8, 9, 10, 11, 12, 13, 14]","[6, 7, 8, 9, 7, 8, 9, 10, 11, 12, 13, 14, 15, ...",0,"[7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 9, 1...","[8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 9, 10, ...","[9, 10, 11, 12, 13, 14, 11, 12, 13, 14, 15, 16...",...,[],[],[],[],[],[],[],[],[],5966
4,5,"[43, 66, 44, 76, 67, 27, 52, 26, 83, 39]","[37, 83, 99, 29, 48, 60, 62, 2, 31, 27, 4, 65,...",4,"[6, 7, 8, 9]","[7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 9, 1...",0,"[8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 9, 10, ...","[9, 10, 11, 12, 13, 14, 11, 12, 13, 14, 15, 16...","[10, 11, 12, 13, 14, 11, 12, 13, 14, 15, 16, 1...",...,[],[],[],[],[],[],[],[],[],2812


In [70]:
print(b['tot'].sum() + len(b['id']))

9496801
